# Analytic estimation of $r$

In [1]:
from utils import *

### Import the decoupled $C_\ell$

We can use the spectra decoupled from the transfer function estimated in the previous section, and select the $B$-mode component to proceed with the analytic estimation of $r$. 

In [2]:
# Import decoupled spectrum
nside = 64
fname_transferval = f'bbmaster/out_CMBl/transfer/transfer_validation.npz'
transfval = np.load(fname_transferval)
cl_dec = transfval['decoupled_cl'] #EE, EB, BE, BB
ecl_dec = transfval['error_decoupled_cl']

In [3]:
# Select only B-mode
cl_bb = cl_dec[3]

### Estimate covariance

To estimate the covariance analytically, we use the ``*Knox formula*'':
$${\sf \Sigma}_\ell^{a,b} \equiv \langle \Delta\hat{{\sf C}}_\ell^{a,b} \Delta\hat{{\sf C}}_\ell^{c,d} \rangle = \delta_{\ell \ell'} \frac{{\sf C}_\ell^{a,c}{\sf C}_\ell^{b,d} + {\sf C}_\ell^{a,d}{\sf C}_\ell^{b,c}}{(2\ell +1)\Delta\ell \, f_{sky}}$$

Here, we assume 10% of the sky, which is compatible with the full SAT observation patch, and $\Delta\ell=1$. 

In [4]:
# Useful defs
nls = len(cl_bb)
ls = np.arange(nls)
fsky = 0.1

In [5]:
# Calculate analytic covariance 
fl = 1./((2*ls+1)*fsky) 
noi = nl_theory(ls) # white noise 
cl_signoi = cl_bb + noi
covar = (2 * cl_signoi * cl_signoi)*fl
cov = np.diag(covar) # [nls, nls]
invcov = np.linalg.inv(cov) # inverse covariance

### Analytic estimate of r

We now proceed to make a ``dirty'' estimate of $r$.

To do so, we can minimize a standard $\chi^2$ of the form
${\left(C_\ell^{\text{data}} - C_\ell^{\text{theory}}\right)}^T {\Sigma_\ell^{-1}} {\left(C_\ell^{\text{data}} - C_\ell^{\text{theory}}\right)} $.

The theory is estimated as the sum of the tensor and the lensing components of the CMB: $ C_\ell^{\text{theory}} = r C_\ell(r = 1) + A_{\text{lens}} C_\ell^{\text{lens}} $

Assuming a single frequency observation of a CMB-only sky, with no $E$-$B$ contaminations in the observed patch of the sky (e.g. from the mask or filterings, which we have supposedly taken care of in the previous Section), the tensor-to-scalar ratio can be recovered analytically as:
$$r_{\text{best fit}} = \frac{{\mathbf{C}_{\ell,\text{primordial}}^T \cdot \mathbf{C}^{-1} \cdot (\mathbf{d} - \mathbf{A}_{\text{lens}} \cdot \mathbf{C}_{\ell,\text{lensing}})}}{{\mathbf{C}_{\ell,\text{primordial}}^T \cdot \mathbf{C}_\ell^{-1} \cdot \mathbf{C}_{\ell,\text{primordial}}}}$$
$$\sigma(r) = \frac{1}{\sqrt{\mathbf{C}_{\text{primordial}_\ell}^T \cdot \mathbf{C}^{-1} \cdot \mathbf{C}_{\text{primordial}_\ell}}}$$


In [9]:
# Minimize chi^2
res = minimize(chi2, [0.], method='Powell', args=(cl_bb))
# Define CMB components
cl_d = cl_bb 
cl_t = (cl_theory(res.x, A_lens=1.)).flatten()
cl_l = (cl_theory(0., A_lens=1.)).flatten()
cl_p = (cl_theory(1., A_lens=0.)).flatten()
num = np.transpose(cl_p).dot(invcov).dot(cl_d - cl_l)
den = np.transpose(cl_p).dot(invcov).dot(cl_p)

In [11]:
# Best fit r, sigma(r)
r_best_fit = num / den
print(r_best_fit) 
sigma_r = 1./np.sqrt(den) 
print(sigma_r)

0.0001342320925694566
0.00023371567435623902
